In [55]:
import nltk
print('hello world')

hello world


In [56]:
def foo():
    print('foo')

In [57]:
foo()

foo


In [58]:
import pandas
#pandas.read_excel(r'C:\Users\n1555085\Downloads\Copy of CSAT Help hub April Responses.xlsx')
sheet = pandas.read_excel(r'C:\Users\n1555085\Downloads\May Help Hub Data.xlsx')
courtesy = sheet['Unnamed: 9'].dropna().values.tolist()[1:]
effectiveness = sheet['Unnamed: 10'].dropna().values.tolist()[1:]
timeliness = sheet['Unnamed: 11'].dropna().values.tolist()[1:]
understanding = sheet['Unnamed: 12'].dropna().values.tolist()[1:]
nps = sheet['Unnamed: 13'].dropna().values.tolist()[1:]
comments = courtesy + effectiveness + timeliness + understanding + nps
date = sheet['Unnamed: 1'].dropna().values.tolist()[1:]
#should add completion rate

In [59]:
print(f'# of courtesy comments: {len(courtesy)}')
print(f'# of effectiveness comments: {len(effectiveness)}')
print(f'# of timeliness comments: {len(timeliness)}')
print(f'# of understanding comments: {len(understanding)}')
print(f'# of nps comments: {len(nps)}')
print(f'# of comments: {len(comments)}')

# of courtesy comments: 10
# of effectiveness comments: 17
# of timeliness comments: 24
# of understanding comments: 13
# of nps comments: 148
# of comments: 212


In [60]:
print('hi')

hi


In [200]:
#read from corpus, remove stopwords
from nltk.corpus import stopwords
from string import punctuation

with open(r'C:\Users\n1555085\Downloads\Project\positiveComments.txt', 'r') as f:
    posReviews = f.readlines()
with open(r'C:\Users\n1555085\Downloads\Project\negativeComments.txt', 'r') as f:
    negReviews = f.readlines()
print(len(posReviews), len(negReviews))

sw = set(stopwords.words('english') + list(punctuation))
notStopwords = ['not', 'no', '!', 'but', 'too', 'have', 'had']

def removeStopwords(review):
    #review.translate(None, string.punctuation)
    return ' '.join([word for word in review.split() if word.lower() not in sw or word.lower() in notStopwords])
posReviews = list(filter(lambda s: s , list(map(removeStopwords, posReviews))))
negReviews = list(filter(lambda s: s , list(map(removeStopwords, negReviews))))

posReviews[0:5]
#negReviews[0:5]

4553 1574


['Spenser Cameron set assisted issues. exceptional trying assist newcomer new systems.',
 'second consultant awesome appropriate follow well letting know break fix consultants suggestion since could not receive emails. best.',
 'support personnel helpful patient issue',
 'fast service.',
 'Greeting robotic, however loosened conversation went on.']

In [176]:
#define pos and neg bag-of words, and vocabulary

posWords = [word.lower() for review in posReviews for word in review.split()]
negWords = [word.lower() for review in negReviews for word in review.split()]
vocabulary = list(set(posWords + negWords))
#vocabulary[0:100]

In [177]:
#define training data: list of (review list of words, label) tuples
trainingData = [(r.split(), 'pos') for r in posReviews] + [(r.split(), 'neg') for r in negReviews]

In [178]:
#NAIVE BAYES CLASSIFIER: extract feature vector from each review, train classifier
def featureVector(reviewSplit):
    reviewWords = set(reviewSplit)
    features = {}
    for word in vocabulary:
        features[word] = word in reviewWords
    return features

naiveBayesClassifier = nltk.NaiveBayesClassifier.train(nltk.classify.apply_features(featureVector, trainingData))

#classify functions
def NBclassify(review):
    review = removeStopwords(review)
    features = featureVector(review.split())
    probDist = naiveBayesClassifier.prob_classify(features)
    confidence = max(probDist.prob('pos'), probDist.prob('neg'))
    #pos and neg add to 1
    return (naiveBayesClassifier.classify(features).upper(), confidence)

def NBclassifyComments(comments):
    return [(c, NBclassify(c)) for c in comments]

In [179]:
NBlabeledComments = NBclassifyComments(effectiveness)

In [180]:
#SVM Classifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

#vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.8, sublinear_tf = True, use_idf = True, ngram_range=(1, 2))
vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
train_vectors = vectorizer.fit_transform(posReviews + negReviews)
labelsList = ['pos'] * len(posReviews) + ['neg'] * len(negReviews)

classifier_linear = SVC(kernel='linear', probability = True)
classifier_linear.fit(train_vectors, labelsList)

classifier_poly = SVC(kernel='poly', probability = True)
classifier_poly.fit(train_vectors, labelsList)

classifier_rbf = SVC(kernel='rbf', probability = True)
classifier_rbf.fit(train_vectors, labelsList)

classifier_sigmoid = SVC(kernel='sigmoid', probability = True)
classifier_sigmoid.fit(train_vectors, labelsList)

# svm kernel can be ‘linear’, ‘poly’, ‘rbf’, or ‘sigmoid’
def SVMclassify(review, kernel):
    review = removeStopwords(review)
    review_vector = vectorizer.transform([review]) # vectorizing
    if kernel == 'linear':
        return (classifier_linear.predict(review_vector)[0], max(classifier_rbf.predict_proba(review_vector)[0]))
    elif kernel == 'poly':
        return (classifier_poly.predict(review_vector)[0], max(classifier_rbf.predict_proba(review_vector)[0]))
    elif kernel == 'rbf':
        return (classifier_rbf.predict(review_vector)[0], max(classifier_rbf.predict_proba(review_vector)[0]))
    elif kernel == 'sigmoid':
        return (classifier_sigmoid.predict(review_vector)[0], max(classifier_rbf.predict_proba(review_vector)[0]))
    return None

def SVMclassifyComments(comments, kernel):
    return [(c, SVMclassify(c, kernel)) for c in comments]

In [231]:
#Labelling new data
SVMlabelledComments = SVMclassifyComments(comments, 'rbf')
labelledPos = []
labelledNeg = []
#lc is (<comment>, (<pos/neg>, <confidence>))
for lc in SVMlabelledComments:
    if lc[1][0] == 'pos':
        labelledPos.append(lc[0])
    elif lc[1][0] == 'neg':
        labelledNeg.append(lc[0])
#remove stopwords
labelledPos = list(filter(lambda s: s , list(map(removeStopwords, labelledPos))))
labelledNeg = list(filter(lambda s: s , list(map(removeStopwords, labelledNeg))))

In [233]:
#Clustering labelled data
from sklearn.cluster import KMeans
import numpy as np

tfidf = TfidfVectorizer(min_df = 5, max_df = 0.8, sublinear_tf = True, use_idf = True, ngram_range=(1, 2))
x = tfidf.fit_transform(labelledPos)
km = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)
km.fit(x)
np.unique(km.labels_, return_counts = True)
text = {}
for i,cluster in enumerate(km.labels_):
    posComment = labelledPos[i]
    if cluster not in text.keys():
        text[cluster] = posComment
    else:
        text[cluster] += posComment

        
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest

keywords = {}
counts = {}
for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(10, freq, key=freq.get)
    counts[cluster] = freq

uniqueKeys = {}
for cluster in range(3):
    other_clusters = list(set(range(3)) - set([cluster]))
    keys_other_clusters = set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique = set(keywords[cluster]) - keys_other_clusters
    uniqueKeys[cluster] = nlargest(10, unique, key=counts[cluster].get)

    

Initialization complete
Iteration 0, inertia 275.21408166494314
Iteration 1, inertia 130.30789571082016
Iteration 2, inertia 130.2186164990933
Converged at iteration 2: strict convergence.


In [241]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

def get_topics(components, feature_names, n=50):
    for idx, topic in enumerate(components):
        print("\nTopic %d: " % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        
v = TfidfVectorizer(max_features=1000) 
X = v.fit_transform(labelledPos)

nmf_model = NMF(n_components=1, init='random', random_state=0)
nmf_top = nmf_model.fit_transform(X)

terms = v.get_feature_names() 
get_topics(nmf_model.components_,terms)


'''from wordcloud import WordCloud
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt

for idx, topic in enumerate(nmf_model.components_):
    if idx == 0:
        topic_x = [(terms[i], topic[i].round(2)) for i in topic.argsort()[:-1000 - 1:-1]]
        topic_x = {i[0]:i[1] for i in topic_x}
            
wordcloud = WordCloud(width = 3000, height = 3000, stopwords=STOPWORDS, background_color="white", min_font_size = 30)
wordcloud = wordcloud.generate_from_frequencies(topic_x)

plt.axis("off")
plt.figure(figsize=(50, 50))
plt.imshow(wordcloud, interpolation="bilinear")
plt.show()'''


Topic 1:  [('quick', 1.68), ('response', 1.17), ('efficient', 1.07), ('resolution', 0.9), ('issue', 0.72), ('resolved', 0.4), ('solution', 0.35), ('fix', 0.31), ('help', 0.29), ('easy', 0.23), ('friendly', 0.21), ('clear', 0.21), ('follow', 0.2), ('solved', 0.19), ('fixed', 0.19), ('problem', 0.18), ('great', 0.18), ('rapid', 0.18), ('expertise', 0.18), ('via', 0.17), ('thanks', 0.16), ('had', 0.16), ('quickly', 0.16), ('timely', 0.14), ('pleasant', 0.14), ('service', 0.14), ('etc', 0.14), ('would', 0.13), ('time', 0.13), ('helpful', 0.13), ('support', 0.13), ('but', 0.12), ('email', 0.12), ('call', 0.12), ('assistance', 0.12), ('took', 0.11), ('nice', 0.11), ('first', 0.11), ('ticket', 0.11), ('swift', 0.1), ('contacted', 0.1), ('quicker', 0.1), ('knew', 0.1), ('fast', 0.1), ('without', 0.09), ('provided', 0.09), ('understood', 0.09), ('understand', 0.09), ('proactive', 0.09), ('completely', 0.09)]


'from wordcloud import WordCloud\nfrom wordcloud import STOPWORDS\nimport matplotlib.pyplot as plt\n\nfor idx, topic in enumerate(nmf_model.components_):\n    if idx == 0:\n        topic_x = [(terms[i], topic[i].round(2)) for i in topic.argsort()[:-1000 - 1:-1]]\n        topic_x = {i[0]:i[1] for i in topic_x}\n            \nwordcloud = WordCloud(width = 3000, height = 3000, stopwords=STOPWORDS, background_color="white", min_font_size = 30)\nwordcloud = wordcloud.generate_from_frequencies(topic_x)\n\nplt.axis("off")\nplt.figure(figsize=(50, 50))\nplt.imshow(wordcloud, interpolation="bilinear")\nplt.show()'